<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/Tensorflow/tf_dataset_tutorial/Exercise_tfds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 3.8MB 50.2MB/s 
     |████████████████████████████████| 450kB 43.3MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ./gdrive/My\ Drive/Deep_learning/Paper_implementation/VGG
!ls

/content/gdrive/My Drive/Deep_learning/Paper_implementation/VGG
checkpoints  Exercise_tfds.ipynb  logs


### Load Library

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import os, math
from tensorflow.keras import layers, datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from datetime import datetime
from tqdm import tqdm_notebook
from glob import glob
from PIL import Image
%matplotlib inline
print(tf.__version__)

2.0.0


### Hyperparameter

In [0]:
num_epoch = 5
batch_size = 32
input_shape = (32, 32, 3)
learning_rate = 0.001
dropout_rate = 0.25
num_classes = 10
initial_epoch = 0

loss_ftn = tf.keras.losses.sparse_categorical_crossentropy
metrics = ['accuracy']
optm = tf.keras.optimizers.Adam(learning_rate)

### Model

In [6]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(96, 3, 1, 'SAME', activation='relu')(inputs)
net = layers.MaxPool2D(2)(net)
net = layers.Conv2D(256, 3, 1, 'SAME', activation='relu')(net)
net = layers.MaxPool2D(2)(net)
net = layers.Conv2D(384, 3, 1, 'SAME', activation='relu')(net)
net = layers.Conv2D(384, 3, 1, 'SAME', activation='relu')(net)
net = layers.Conv2D(256, 3, 1, 'SAME', activation='relu')(net)
net = layers.MaxPool2D(2)(net)
net = layers.Flatten()(net)
net = layers.Dense(1024)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(1024)(net)
net = layers.Activation('relu')(net)
net = layers.Dense(10)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='My_AlexNet')
model.compile(optimizer=optm, loss=loss_ftn, metrics=metrics)
model.summary()

Model: "My_AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 96)        2688      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       221440    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 384)         885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 384)         1

### Callbacks

In [0]:
# Tensorboard
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard = tf.keras.callbacks.TensorBoard(
    logdir,
    histogram_freq=1,
    write_graph=True,
    write_images=True
)

# Learning rate scheduler
def scheduler(epoch):
  if epoch < 10: return 0.001
  else: return 0.001 * math.exp(0.1 * (10 - epoch))
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

# Checkpoints
ckptdir = 'checkpoints'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    ckptdir,
    monitor='val_accuracy',
    verbose=1,
    save_best_inly=True
)

# callbacks = [tensorboard, learning_rate_scheduler, checkpoint]
callbacks = [learning_rate_scheduler]

### Data load

In [8]:
(train_ds, test_ds), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete3IH91V/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete3IH91V/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


### Define Augmentation ftn

In [0]:
def normalize_img(image, label):
  return tf.cast(image, dtype=tf.float32) / 255., label

def image_preprocess(image, label):
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  image = tf.image.random_brightness(image, 0.2)
  return image, label

### Make dataset

In [0]:
trian_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.map(image_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(ds_info.splits['train'].num_examples)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
train_ds = train_ds.repeat()

test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.repeat()

### Training

In [11]:
steps_per_epoch = ds_info.splits['train'].num_examples // batch_size
validation_steps = ds_info.splits['test'].num_examples // batch_size
steps_per_epoch, validation_steps

(1562, 312)

In [12]:
history = model.fit(
    train_ds,
    epochs=num_epoch,
    callbacks=callbacks,
    validation_data=test_ds,
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

Train for 1562 steps, validate for 312 steps

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/5
1562/1562 [==============================] - 33s 21ms/step - loss: 1.9447 - accuracy: 0.2977 - val_loss: 4.3584 - val_accuracy: 0.1007

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Epoch 2/5
1562/1562 [==============================] - 15s 9ms/step - loss: 1.5487 - accuracy: 0.4308 - val_loss: 4.1418 - val_accuracy: 0.1002

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
Epoch 3/5
1562/1562 [==============================] - 15s 9ms/step - loss: 1.4131 - accuracy: 0.4847 - val_loss: 4.1995 - val_accuracy: 0.1002

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.
Epoch 4/5
1562/1562 [==============================] - 15s 9ms/step - loss: 1.3098 - accuracy: 0.5263 - val_loss: 5.0239 - val_accuracy: 0.1004

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.
Epoch 5/5
1562/1562 [============